# Exploration Données de Kronyx67

### Importation

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np


### Téléchargement des données 

In [4]:
df = pd.read_csv("../data/Kronyx67.csv")

# 2. Aperçu rapide
df.head() 

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid
0,1764516892,"30 Nov 2025, 15:34",Nava,3654dbc1-1f71-4686-8a9d-c8d63f416598,Tic Tac,NaN,Tic Tac,NaN
1,1764516699,"30 Nov 2025, 15:31",Yuston XIII,NaN,Que des cendres,NaN,Que des cendres,NaN
2,1764516456,"30 Nov 2025, 15:27",Mathilde,f7b37e03-d216-4144-901a-cebca6426416,La nuit · Le jour,NaN,Libre !,NaN
3,1764516286,"30 Nov 2025, 15:24",Théadule,NaN,Un Monde Parfait,NaN,Un Monde Parfait,NaN
4,1764516117,"30 Nov 2025, 15:21",Louisette & Amaury,NaN,Mieux,NaN,Mieux,NaN


### Ajout de Colonne supplémentaires

In [5]:
# Convertir la colonne de temps
df["utc_time"] = pd.to_datetime(df["utc_time"], format="%d %b %Y, %H:%M")

# Ajouter des colonnes utiles
df["date"] = df["utc_time"].dt.date
df["year"] = df["utc_time"].dt.year
df["hour"] = df["utc_time"].dt.hour
df["weekday"] = df["utc_time"].dt.day_name()
df["week"] = df["utc_time"].dt.isocalendar().week  # Numéro de semaine ISO

In [6]:
df.head()

,uts,utc_time,artist,artist_mbid,album,album_mbid,track,track_mbid,date,year,hour,weekday,week
0,1764516892,2025-11-30 15:34:00,Nava,3654dbc1-1f71-4686-8a9d-c8d63f416598,Tic Tac,NaN,Tic Tac,NaN,2025-11-30,2025,15,Sunday,48
1,1764516699,2025-11-30 15:31:00,Yuston XIII,NaN,Que des cendres,NaN,Que des cendres,NaN,2025-11-30,2025,15,Sunday,48
2,1764516456,2025-11-30 15:27:00,Mathilde,f7b37e03-d216-4144-901a-cebca6426416,La nuit · Le jour,NaN,Libre !,NaN,2025-11-30,2025,15,Sunday,48
3,1764516286,2025-11-30 15:24:00,Théadule,NaN,Un Monde Parfait,NaN,Un Monde Parfait,NaN,2025-11-30,2025,15,Sunday,48
4,1764516117,2025-11-30 15:21:00,Louisette & Amaury,NaN,Mieux,NaN,Mieux,NaN,2025-11-30,2025,15,Sunday,48


In [7]:
# --- Option : sélectionner une année spécifique ---
year_selected = 2025
df_year = df[df["year"] == year_selected]

jours_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# --- Regrouper par semaine et jour ---
heatmap_data = df_year.groupby(['week', 'weekday']).size().reset_index(name='plays')

# Pivot pour créer la matrice
matrix = heatmap_data.pivot(index='weekday', columns='week', values='plays').reindex(jours_order)

# Remplacer les 0 par NaN pour laisser les cases vides
matrix = matrix.replace(0, np.nan)

# --- Étiquettes pour axes ---
semaines = [f"W{w}" for w in matrix.columns]
jours = matrix.index.tolist()

# --- Création de la heatmap ---
fig = px.imshow(
    matrix.values,
    x=semaines,
    y=jours,
    text_auto=True,
    color_continuous_scale='Turbo'
)

# --- Mettre le fond blanc pour les NaN ---
fig.update_traces(
    hovertemplate="%{y}, %{x}: %{z}<extra></extra>",
    zmin=0
)
fig.update_layout(
    title=f"Weekly activity per week - Year {year_selected}",
    xaxis_title="Week of the year",
    yaxis_title="Day of the week",
    plot_bgcolor='white',   # fond de la zone de tracé
    paper_bgcolor='white'   # fond autour du graphique
)

fig.show()

In [9]:

artist_counts = df["artist"].value_counts().reset_index()
artist_counts.columns = ["artist", "plays"]
fig = px.bar(
    artist_counts.head(10),
    x="plays",
    y="artist",
    orientation="h",
    title="Top 10 artistes écoutés",
    color="plays",
    color_continuous_scale="viridis"
)
fig.show()


In [10]:
hourly = df.groupby("hour").size().reset_index(name="plays")
px.area(hourly, x="hour", y="plays", title="Répartition des écoutes par heure", markers=True)


In [12]:
# On agrège les données pour compter le nombre d'écoutes par Jour et par Heure
heatmap_data = df.groupby(['weekday', 'hour']).size().reset_index(name='count')

fig = px.density_heatmap(heatmap_data, 
                         x="hour", 
                         y="weekday", 
                         z="count", 
                         title="🔥 Intensité d'écoute : Jours vs Heures",
                         labels={'weekday': 'Jour', 'hour': 'Heure', 'count': 'Écoutes'},
                         color_continuous_scale="Viridis")

fig.update_layout(xaxis_dtick=2) # Affiche une heure sur deux pour la lisibilité
fig.show()

In [13]:
# On groupe par Année et Semaine pour éviter de mélanger les années si le fichier est long
weekly_stats = df.groupby(['year', 'week']).size().reset_index(name='count')

# Création d'une colonne combinée pour l'affichage (ex: "2025 - Week 48")
weekly_stats['Week_Label'] = weekly_stats['year'].astype(str) + " - Semaine " + weekly_stats['week'].astype(str)

fig = px.bar(weekly_stats, 
             x='Week_Label', 
             y='count',
             title="📅 Volume d'écoute par Semaine",
             labels={'Week_Label': 'Semaine', 'count': 'Nombre de titres'},
             color='count',
             color_continuous_scale='Magma')

fig.show()

In [14]:
# On crée une colonne unique "Titre - Artiste" pour l'affichage
df['Full_Track'] = df['track'] + " - " + df['artist']

top_tracks = df['Full_Track'].value_counts().head(10).reset_index()
top_tracks.columns = ['Titre', 'Ecoutes']

fig = px.bar(top_tracks, 
             x='Ecoutes', 
             y='Titre', 
             orientation='h', # Barres horizontales
             title="🎵 Top 10 des Titres les plus écoutés",
             color='Ecoutes',
             color_continuous_scale='Teal')

fig.update_layout(yaxis={'categoryorder':'total ascending'}) # Trie du plus grand au plus petit
fig.show()

In [15]:
# On prend le top 30 pour que le graphique reste lisible
top_30_artists = df['artist'].value_counts().head(30).index
df_tree = df[df['artist'].isin(top_30_artists)]

# Compte par Artiste et Album
df_tree_grouped = df_tree.groupby(['artist', 'album']).size().reset_index(name='count')

fig = px.treemap(df_tree_grouped, 
                 path=[px.Constant("Ma Musique"), 'artist', 'album'], 
                 values='count',
                 title="💽 Carte des albums favoris (Treemap)",
                 color='count',
                 color_continuous_scale='RdBu')

fig.show()